In [11]:
import os
import torch

In [20]:
model_parent_path=''#os.getcwd()
model_path='google-t5/abc.pkl'
model_state_dict_path='/work/cvp352/loi_work/loi_research/GenMC/results/obqa_large/lr_0.0001_seed_1_bs_8_ga_1_layer_num_1_alpha_0.5_beta_1'

# model_name='allenai/unifiedqa-v2-t5-large-1363200'
model_name=None
tokenizer_name='google-t5/t5-large'#model_name #'google-t5/t5-large'
max_answer_length = 300
BATCH_SIZE=10
accumulate_step = None
DATA_NAME_SINGLE='obqa_fact'
NUM_EPOCHS=1
VISIBLE_DEVICE=','.join([str(x) for x in range(torch.cuda.device_count())])
DEVICE = 0

In [13]:
os.environ['CUDA_VISIBLE_DEVICES'] = VISIBLE_DEVICE #cannot work
print(f"Loi params: {model_path}{'#'*5}{max_answer_length}{'#'*5}batch_size={BATCH_SIZE};accumulate_step={accumulate_step}")
DATABASE_NAME = [DATA_NAME_SINGLE]#,'arc_hard']#,'race','mctest',]


Loi params: work/cvp352/loi_work/loi_research/GenMC/results/obqa_large/lr_0.0001_seed_1_bs_8_ga_1_layer_num_1_alpha_0.5_beta_1#####300#####batch_size=10;accumulate_step=None


In [14]:
import torch
import pickle
from torch.utils.data import Dataset, DataLoader
from tqdm import trange
from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor
from functools import wraps, partial
from torch.nn.modules.sparse import Embedding
from torch.optim import Adam, SGD
import torch.nn as nn
from tqdm import tqdm
from random_utils import set_seed
import pandas as pd

In [23]:
# model_name = "loi_with_padding_just_same_answer_len_70.pkl"

# "loi_with_padding_1.pkl"#
# model_name = (
#     "allenai/unifiedqa-v2-t5-large-1363200"  # you can specify the model size here
# )
tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)
if model_state_dict_path:
    a=torch.load(model_state_dict_path)['model_state_dict']
    model_original=T5ForConditionalGeneration.from_pretrained('t5-large', device_map='auto')
    model_original.load_state_dict(a)
else:
    model_original = T5ForConditionalGeneration.from_pretrained(
        f'{model_parent_path}/{model_path}' if model_path else model_name, device_map='auto')  # 'auto')
model = model_original
# model.hf_device_map

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


IsADirectoryError: [Errno 21] Is a directory: '/work/cvp352/loi_work/loi_research/GenMC/results/obqa_large/lr_0.0001_seed_1_bs_8_ga_1_layer_num_1_alpha_0.5_beta_1'

In [ ]:
NUM_QUESTION,EACH_HAS,dataset_test=pickle.load(
        open(f"multiple_choice_datasets/{DATA_NAME_SINGLE}_test_permute.pkl", "rb"))
print(NUM_QUESTION, EACH_HAS)

In [ ]:
import textwrap

# ques=4
print(textwrap.fill(dataset_test[0][0]))
# print('correct: ',dataset_test[ques][1],'length',len(dataset_test[0][0]))


In [ ]:
def measure_unalike(arr):
    n = len(arr)
    arr = pd.Series(arr).value_counts()
    return 1 - ((arr / n)**2).sum()



question_to_do = 5
per_question = 20


def get_model_forward(input_ids, attention_mask, model=model):
    with torch.no_grad():
        start = []
        [start.append([0]) for x in range(len(input_ids))]
        for k in range(max_answer_length):
            # print(torch.tensor(start).shape)
            result = model(
                input_ids=input_ids.to(DEVICE),
                attention_mask=attention_mask.to(DEVICE),
                decoder_input_ids=torch.tensor(start).to(DEVICE),
                output_attentions=True,
            )
            item = result.logits.argmax(dim=2)[:, -1]
            # print('loi',result.logits.shape, item)
            for index in range(len(item)):
                start[index].append(item[index].item())
            if torch.allclose(item, torch.tensor(1)):
                break
            #     break
    result = []
    for batch in start:
        y = -1
        for index, x in enumerate(batch):
            if x == 1:
                y = index
                break
        result.append(batch[:y+1] if y > -1 else batch)
    return [tokenizer.decode(x, skip_special_tokens=True) for x in result]


def run_model(input_strs):
    if input_strs is str:
        input_strs = [input_strs]
    input_ids_wrapper = tokenizer(
        input_strs, padding=True, return_tensors='pt')

    answer = get_model_forward(input_ids_wrapper['input_ids'],
                               input_ids_wrapper['attention_mask'])
    return answer

In [ ]:
run_model(dataset_test[0][0])

In [ ]:
print(NUM_QUESTION, EACH_HAS)

In [ ]:
def evaluate():
    wrong_answers=[]
    got_2=[]
    got_1=[]
    answers=[]
    last_str=None
    last_acc=None
    groups=[[] for x in range(NUM_QUESTION)]
    correct=[False for x in range(NUM_QUESTION)]
    count = 0
    count1 = 0
    count2 = 0
    count10 = 0
    total = 0
    data=dataset_test
    pbar1 = trange(len(data))
    for idx, ques in enumerate(pbar1):
        groupid=int(idx /EACH_HAS)
        question = data[ques][0]
        key = data[ques][1]
        total += 1
        answer = run_model(question)[0]
        groups[groupid].append(answer)
        answers.append(answer)
        if key == answer:
            count += 1
            correct[groupid]|=True
        else:
            wrong_answers.append(ques)
        if key[0] == answer[0]:
            count1 += 1
            got_1.append(ques)
        if key[:2] == answer[:2]:
            count2 += 1
            got_2.append(ques)
        if answer in question:
            count10 += 1
        last_str=f"{count}, {count1}, {count2}, {count10},{total},{count/total*100:.2f},{count10/total*100:.2f}"
        last_acc=f'{count/total*100:.2f}'
        pbar1.set_postfix_str(last_str)
    return groups, correct, answers
groups, correct, anwers=evaluate()

### Measure consistency

In [ ]:
consistency=[0 for _ in range(len(groups))]
for idx in range(len(groups)):
    consistency[idx]=1-measure_unalike(groups[idx])
    print(idx, correct[idx], consistency[idx])
    
print(f'Avg {sum(consistency)/len(consistency)}')

### Measure Recall STD

In [ ]:
how_many=len(dataset_test[0][0].split(' ( ) ')[1:])
total_loc=[[0 for _ in range(how_many)] for _ in range(NUM_QUESTION)]
total_acc=[[0 for _ in range(how_many)] for _ in range(NUM_QUESTION)]
for ques in range(len(dataset_test)):
    groupid=int(ques /EACH_HAS)
    key=dataset_test[ques][1]
    answers_from_text=dataset_test[ques][0].split(' ( ) ')[1:]
    location=answers_from_text.index(key)
    total_loc[groupid][location]+=1
    if anwers[ques]==key:
        total_acc[groupid][location]+=1
#         if groups[groupid][ques]==data[ques][1]:
                
# for groupid in range(len(groups)):
    

# groups[0]

### Micro

In [ ]:
# import numpy as np


# recall_rate=[[x/y*100 for x,y in zip(total_acc[k], total_loc[k])] for k in range(NUM_QUESTION)]
# print(recall_rate)
# each_ques=[np.std(x) for x in recall_rate]
# print('Each ',each_ques)
# print('Final mean std',np.mean(each_ques))
# # print('std ', np.std(recall_rate))

### Macros

In [ ]:
import numpy as np


print('Total',np.array(total_loc).sum(axis=0))
print('each',(np.array(total_acc).sum(axis=0)/np.array(total_loc).sum(axis=0)*100))
(np.array(total_acc).sum(axis=0)/np.array(total_loc).sum(axis=0)*100).std()